# Real world Data

In this dataset, we have the production of a numerous amount of Chinese firm by city and industry. In order the create a balanced panel at the city-industry-year level, we need to create a squared matrix. It means, we need to have a dataframe with all the industries available by city and year. 

## Objective:

- Create a balanced panel data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataPandasClass_china.gz")
df.shape

(63907, 6)

In [3]:
df.head()

,year,geocode4_corr,cityen,SOE,indu_2,output
0,2002,1101,Beijing,Private,13,3318123.0
1,2002,1101,Beijing,Private,14,4805891.0
2,2002,1101,Beijing,Private,15,3997092.0
3,2002,1101,Beijing,Private,17,2743933.0
4,2002,1101,Beijing,Private,18,3797050.0


In [21]:
df['cityen'].nunique()

270

In [23]:
df['year'].nunique()

7

In [24]:
df['indu_2'].nunique()

29

In [25]:
7 * 270 * 29 * 2

109620

In [14]:
df['SOE'].unique()

array(['Private', 'SOE'], dtype=object)

In [15]:
df.sort_values(['year', 'cityen', 'indu_2', 'SOE']).head()

,year,geocode4_corr,cityen,SOE,indu_2,output
9506,2002,6109,Ankang,SOE,13,12914.0
9497,2002,6109,Ankang,Private,14,28768.0
9507,2002,6109,Ankang,SOE,14,0.0
9508,2002,6109,Ankang,SOE,15,522.0
9509,2002,6109,Ankang,SOE,16,475935.0


In [16]:
to_group_reshape = [

    'cityen',
    'year', 'indu_2', 'SOE'
]

In [17]:
df_drop = df.drop(columns = 'geocode4_corr')
df_drop.head()

,year,cityen,SOE,indu_2,output
0,2002,Beijing,Private,13,3318123.0
1,2002,Beijing,Private,14,4805891.0
2,2002,Beijing,Private,15,3997092.0
3,2002,Beijing,Private,17,2743933.0
4,2002,Beijing,Private,18,3797050.0


In [27]:
df_drop.pivot_table(index = ['cityen', 'indu_2', 'SOE'],
         columns ='year',
         #values = 'output'
        ).fillna(0).stack()

output
cityen indu_2 SOE     year          
Ankang 13     Private 2002       0.0
                      2003   73235.0
                      2004  144069.0
                      2005  193215.0
                      2006  265348.0
...                              ...
Zunyi  42     SOE     2004  279099.0
                      2005  143177.0
                      2006  168620.0
                      2007  149810.0
                      2008       0.0

[83531 rows x 1 columns]

In [ ]:
df_drop.to_csv('test.csv')

In [9]:
df_perc = (df_drop
           .groupby(['indu_2', 'SOE',  'year'])
           .sum()
           .groupby(level = [0, 1])
           .pct_change().reset_index()
          )
#).stack()
df_perc.head()

,indu_2,SOE,year,output
0,13,Private,2002,NaN
1,13,Private,2003,0.327197
2,13,Private,2004,0.411001
3,13,Private,2005,0.284361
4,13,Private,2006,0.247106


In [ ]:
import seaborn as sns

In [ ]:
sns.catplot(x="year",
            y="output",
            col="indu_2",
            data=df_perc.loc[lambda x:x['SOE'] == 'SOE'], 
            kind="bar",
            col_wrap=5,
           )

In [ ]:
sns.catplot(x="SOE",
            y="output",
            col="indu_2",
            hue = 'year',
            data=df_perc, 
            kind="bar",
            col_wrap=5,
           )

In [ ]:
(df
 .set_index(to_group_reshape)
 .unstack(-1, fill_value=0)
 .stack()
 .unstack(-2, fill_value=0)
 .stack()
 .reset_index()
 )
